# Env setup

In [122]:
%load_ext autoreload

%autoreload 2

In [1]:
import boto3
import pandas as pd
import numpy as np
import configparser
import psycopg2
from faker import Faker
import constants as cons

# Reading configuration file and credentials

In [2]:
config = configparser.ConfigParser()

config.read(cons.CONFIG_FILE)

['config.cfg']

In [3]:
aws_rds_conn = boto3.client('rds', aws_access_key_id=config.get(cons.USER, cons.ACCESS_KEY),
                            aws_secret_access_key=config.get(cons.USER, cons.SECRET_KEY),
                            region_name='us-east-1'
                            )

# Checking available instances for user

In [4]:
rds_instances_ids = []
aws_response = aws_rds_conn.describe_db_instances()

for db_instance in aws_response['DBInstances']:
    db_instance_identifier = db_instance['DBInstanceIdentifier']
    db_engine = db_instance['Engine']
    if (db_engine == 'mysql'):
      aws_mysql = db_instance
    else:
      aws_postgres = db_instance
    print(f"La instancia '{db_instance_identifier}' utiliza el motor de base de datos '{db_engine}'")


La instancia 'ja3plqub98cr' utiliza el motor de base de datos 'mysql'
La instancia 'xux3br0jo1lp' utiliza el motor de base de datos 'postgres'


In [5]:
print(aws_mysql['Endpoint']['Address'])
print(aws_postgres['Endpoint']['Address'])

ja3plqub98cr.cr0suguo232c.us-east-1.rds.amazonaws.com
xux3br0jo1lp.cr0suguo232c.us-east-1.rds.amazonaws.com


In [21]:
try: 
    response = aws_rds_conn.create_db_instance(
        DBInstanceIdentifier=config.get(cons.DB, cons.INSTANCE_ID),
        DBName=config.get(cons.DB, cons.DB_NAME),
        MasterUsername=config.get(cons.DB, cons.DB_USERNAME),
        MasterUserPassword=config.get(cons.DB, cons.DB_PASSWORD),
        Port=int(config.get(cons.DB, cons.PORT)),
        DBInstanceClass= cons.INSTANCE_CLASS,
        Engine=cons.ENGINE,
        PubliclyAccessible=True,
        AllocatedStorage=cons.ALLOC_STORAGE,
        VpcSecurityGroupIds=[config.get(cons.VPC, cons.SEC_GROUP)]
    )
except aws_rds_conn.exceptions.DBInstanceAlreadyExistsFault:
    print("La instancia ya existe")
except Exception as ex:
    print("Error!! ", ex)

## ETL

### Connection to MySQL

import de connector y confifuracion de la conexion.

In [179]:
import mysql.connector #TODO: Remove, only testing
import psycopg2 #TODO: Remove, only testing

# Establecer la conexión con la base de datos Mysql
conexionMysql = mysql.connector.connect(
		host=aws_mysql['Endpoint']['Address'],  # Cambia esto por la dirección IP o el nombre del host de tu servidor MySQL
		user=config.get('DB-MYSQL', 'DB_MYSQL_USERNAME'),  # Cambia esto por tu nombre de usuario de MySQL
		password=config.get('DB-MYSQL', 'DB_MYSQL_PASSWORD'),  # Cambia esto por tu contraseña de MySQL
		database=config.get('DB-MYSQL', 'DB_MYSQL_NAME') # Cambia esto por el nombre de tu base de datos
)

# Establecer la conexión con la base de datos Postgres
dbname = config.get('DB-POSTGRE', 'DB_POSTGRE_NAME')
user = config.get('DB-POSTGRE', 'DB_POSTGRE_USERNAME') # ! Change for user in local Postgres DB
password = config.get('DB-POSTGRE', 'DB_POSTGRE_PASSWORD')
host = aws_postgres['Endpoint']['Address'] # ! Change for url
port = aws_postgres['Endpoint']['Port']

try:
	conexionPSQL = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)
except:
  print('Error de conexion')

### Probando conexion de AWS

Conexion MYSQL

In [7]:
cursor = conexionMysql.cursor()

# Ejecuta una consulta para mostrar las tablas de la base de datos
cursor.execute("select * from articulo")

# Obtiene los resultados de la consulta
tables = cursor.fetchall()

# Imprime las tablas
print("Tablas en la base de datos:")
for table in tables:
    print(table)

# Cierra el cursor y la conexión
cursor.close()
conexionMysql.close()

Tablas en la base de datos:
(1, 3, '4572570011404', 'Operation', Decimal('489.63'), 843, 'it too hospital such see indicate rate say statement film', 'boy.bmp', 0)
(2, 22, '5534368831952', 'Accept', Decimal('232.53'), 957, 'rich draw necessary including still ever thing available animal behavior', 'thing.jpg', 1)
(3, 9, '5541609645667', 'Manager', Decimal('407.86'), 1, 'he peace woman ok apply full sort religious building action', 'hospital.gif', 0)
(4, 10, '4520433978142', 'Pull', Decimal('490.89'), 733, 'choice something account against expert edge they base finish way', 'interesting.gif', 1)
(5, 5, '5505920218887', 'Foot', Decimal('203.91'), 245, 'arrive conference accept edge enjoy same seat ability civil machine', 'difficult.gif', 0)
(6, 13, '5541326743592', 'Red', Decimal('193.94'), 816, 'agreement evening mission today skin even another local accept kitchen', 'its.png', 0)
(7, 2, '5522476610705', 'Laugh', Decimal('334.56'), 397, 'door admit exist fish food debate magazine action

Test conexion PSQL

In [191]:
cursor = conexionPSQL.cursor()

# Ejecuta una consulta para mostrar las tablas de la base de datos
cursor.execute("SELECT column_name FROM information_schema.columns WHERE table_name = 'dimfechas'")

# Obtiene los resultados de la consulta
tables = cursor.fetchall()

# Imprime las tablas
print("Tablas en la base de datos:")
for table in tables:
    print(table[0])

# Cierra el cursor y la conexión
cursor.close()
conexionPSQL.close()

Tablas en la base de datos:
campo_timestamp
fecha_completa
dia_semana
num_dia_mes
dia_num_total
cuarto_fiscal
anio_fiscal
ultimo_dia_mes_bandera
mismo_dia_anio_anterior_fecha
idfecha
num_semana_anio
num_semana_total
fecha_inicio_semana
fecha_inicio_semana_llave
mes
mes_num_total
cuarto
anio
anio_mes
mes_fiscal
dia_nombre
dia_abrev
semana_bandera
mes_nombre
month_abrev


### Insertando data transaccional

In [175]:
# ! La Data ya fue ingresada
import csv

cursor = conexionMysql.cursor()

try:
	with open('./Proyecto_Final_Data/categoria.csv', 'r') as file:
			csv_data = csv.reader(file)
			next(csv_data)  # Ignora la primera fila (cabecera)

			for row in csv_data:
					cursor.execute("INSERT INTO categoria (idcategoria, nombre, descripcion, estado) VALUES (%s, %s, %s, %s)", row)


	with open('./Proyecto_Final_Data/articulo.csv', 'r') as file:
				csv_data = csv.reader(file)
				next(csv_data)  # Ignora la primera fila (cabecera)

				for row in csv_data:
					cursor.execute("INSERT INTO articulo (idarticulo, idcategoria, codigo, nombre, precio_venta, stock, descripcion, imagen, estado) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)", row)
  
	with open('./Proyecto_Final_Data/persona.csv', 'r') as file:
			csv_data = csv.reader(file)
			next(csv_data)  # Ignora la primera fila (cabecera)

			for row in csv_data:
				cursor.execute("INSERT INTO persona (idpersona, tipo_persona, nombre, tipo_documento, num_documento, direccion, telefono, email) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)", row)

	with open('./Proyecto_Final_Data/rol.csv', 'r') as file:
			csv_data = csv.reader(file)
			next(csv_data)  # Ignora la primera fila (cabecera)

			for row in csv_data:
				cursor.execute("INSERT INTO rol (idrol, nombre, descripcion, estado) VALUES (%s, %s, %s, %s)", row)

	with open('./Proyecto_Final_Data/usuario.csv', 'r') as file:
			csv_data = csv.reader(file)
			next(csv_data)  # Ignora la primera fila (cabecera)

			for row in csv_data:
				cursor.execute("INSERT INTO usuario (idusuario, idrol, nombre,tipo_documento, num_documento, direccion, telefono, email, clave, estado) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", row)

	with open('./Proyecto_Final_Data/ingreso.csv', 'r') as file:
			csv_data = csv.reader(file)
			next(csv_data)  # Ignora la primera fila (cabecera)

			for row in csv_data:
				cursor.execute("INSERT INTO ingreso (idingreso, idproveedor, idusuario, tipo_comprobante, serie_comprobante, num_comprobante, fecha, impuesto, total, estado) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", row)

	with open('./Proyecto_Final_Data/venta.csv', 'r') as file:
			csv_data = csv.reader(file)
			next(csv_data)  # Ignora la primera fila (cabecera)

			for row in csv_data:
				cursor.execute("INSERT INTO venta (idventa, idcliente, idusuario, tipo_comprobante, serie_comprobante, num_comprobante, fecha, impuesto, total, estado) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", row)

	with open('./Proyecto_Final_Data/detalle_venta.csv', 'r') as file:
			csv_data = csv.reader(file)
			next(csv_data)  # Ignora la primera fila (cabecera)

			for row in csv_data:
				cursor.execute("INSERT INTO detalle_venta (iddetalle_venta, idventa, idarticulo, cantidad, precio, descuento) VALUES (%s, %s, %s, %s, %s, %s)", row)

	with open('./Proyecto_Final_Data/detalle_ingreso.csv', 'r') as file:
			csv_data = csv.reader(file)
			next(csv_data)  # Ignora la primera fila (cabecera)

			for row in csv_data:
				cursor.execute("INSERT INTO detalle_ingreso (iddetalle_ingreso, idingreso, idarticulo, cantidad, precio) VALUES (%s, %s, %s, %s, %s)", row)

	# Confirma los cambios en la base de datos
	conexionMysql.commit()

	# Cierra el cursor y la conexión
	
finally:
	cursor.close()
	conexionMysql.close()

### Codigo ETL

In [ ]:
def cerrarConexion():
	conexionMysql.close()
	conexionPSQL.close()
	print("Conexión cerrada")


def extraerData(consulta):
	cursor = conexionMysql.cursor()

	print('Extrayendo data')
	cursor.execute(consulta)

	resultados = cursor.fetchall()

	cursor.close()

	return resultados

def insertarData(resultados, tabla, campos):
	print('Insertando data')
	cursor = conexionPSQL.cursor()
	i=0

	for fila in resultados:
		i = i+1
		campos_str = ", ".join(campos)
		marcadores = ", ".join(["%s"] * len(fila))

		insert_sql = f"""
		INSERT INTO {tabla} ({campos_str})
		VALUES ({marcadores})
		"""
		print('Ejecutando Query ', i)
		cursor.execute(insert_sql, fila)

	conexionPSQL.commit()

	cursor.close() 

def dimensionesSql ():
  print('Creando tablas dimensionales')
  ruta = 'ddl_dimensional.sql'
  with open(ruta, 'r') as archivo:
    consulta = archivo.read()

  cursor = conexionPSQL.cursor()

  cursor.execute(consulta)
  conexionPSQL.commit()

  cursor.close()

def generarDimArticulo ():
  print('Dimension Articulo')
  consulta_sql = """
    SELECT 
			idarticulo,
			categoria.nombre,
			articulo.codigo,
			articulo.nombre,
			articulo.precio_venta
    FROM articulo
    INNER JOIN categoria ON articulo.idcategoria = categoria.idcategoria
    """

  data = extraerData(consulta_sql)
  insertarData(data, 'dimArticulo', ['idarticulo', 'categoria', 'codigo', 'nombre','precio_venta'])

def generarDimPersona ():
  print('Dimension Persona')
  consulta_sql = """
    SELECT 
			idpersona,
			nombre,
			num_documento,
			email
    FROM persona
    """
  
  data = extraerData(consulta_sql)
  insertarData(data, 'dimPersona',['idpersona','nombre','num_documento','email'])

def generarDimVenta ():
  print('Dimension Venta')
  consulta_sql = """
    SELECT 
			venta.idventa,
			idpersona,
			nombre,
			idarticulo,
			cantidad,
			precio,
			descuento,
			fecha,
			total
    FROM venta
		INNER JOIN persona ON venta.idcliente = persona.idpersona
		INNER JOIN detalle_venta on venta.idventa = detalle_venta.idventa
		GROUP BY idarticulo
    """
  
  data = extraerData(consulta_sql)
  insertarData(data, 'dimVenta',['idventa','idpersona','nombreCliente','idarticulo','cantidad', 'precio', 'descuento', 'fecha','total'])

def generarDimFechas ():
	print('Dimension Fechas')
	consulta_sql = """
    SELECT DISTINCT
			fecha
		FROM venta
		"""
	data= extraerData(consulta_sql)
	# Fecha y hora

	dataInsert = []
	for elemento in data:

		fecha_hora = elemento[0]
  
		fecha_entero = int(fecha_hora.strftime('%Y%m%d'))

		# Obtener el día de la semana (0 = lunes, 1 = martes, ..., 6 = domingo)
		dia_semana = fecha_hora.weekday()
  
		num_dia_mes = fecha_hora.day

		# Obtener el nombre del día de la semana
		dia_nombre = fecha_hora.strftime('%A')
  
		mes = fecha_hora.month

		# Obtener el nombre del mes
		mes_nombre = fecha_hora.strftime('%B')

		# Obtener el cuarto del año
		cuarto = (mes - 1) // 3 + 1

		anio = fecha_hora.year
  
		dataInsert.append((fecha_entero,data[0][0], dia_semana, num_dia_mes, dia_nombre, mes, mes_nombre, cuarto, anio))
  
	insertarData(dataInsert, 'dimFechas', ['idfecha','fecha_completa','dia_semana','num_dia_mes','dia_nombre','mes','mes_nombre','cuarto','anio'])

def generarDimensiones():
	#dimArticulo
	generarDimArticulo()

	#dimPersona
	generarDimPersona()

	#dimVenta/Hechos
	generarDimVenta()

	#dimFechas
	generarDimFechas()

	return 0

if conexionMysql.is_connected() and conexionPSQL:
	print("Conexión exitosa a las base de datos, MySQL y PostgreSQL")

	# Realiza operaciones en la base de datos aquí
	generarDimensiones()

	# Cerrar conexion
	cerrarConexion()